In [20]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go

In [21]:
# root = "logs-csv"
root = "/mnt/c/Users/piotr.hondra/Documents/inz/indoor-semantic-segmentation/converted_logs"
filenames = os.listdir(root)
dfs = {filename:pd.read_csv(os.path.join(root, filename)) for filename in filenames}


In [22]:
macro_values=[]
segmentation_values=[]
classification_values =[]

for name, df in dfs.items():
    print(20*'#', name)
    test_df = df[df["metric"].str.contains("test.+")]
    class_agg_df = test_df.join(
        test_df["metric"].str.extract(
            "(Accuracy|F1Score).+(other_indoor|kitchen|bedroom|living_room|bathroom|office|dining_room)"
        )
    )[[0, 1, "value"]].rename(columns={0:"metric_name", 1: "class_name"}).dropna()

    seg_agg_df = test_df.join(
        test_df["metric"].str.extract(
            "(Accuracy|Jaccard).+(window|bed$|books|ceiling|chair|floor|furniture|objects|picture|sofa|table|tv|wall)"
        )
    )[[0, 1, "value"]].rename(columns={0:"metric_name", 1: "class_name"}).dropna()

    # print(agg_df)
    # print(seg_agg_df)
    macro_agg_df = test_df.join(
        test_df["metric"].str.extract(
            "macro.+(Accuracy|Jaccard|F1Score)"
        )
    )[[0, "value"]].rename(columns={0:"metric_name",}).dropna()


    segmentation_values.append(seg_agg_df)
    classification_values.append(class_agg_df)
    macro_values.append(macro_agg_df)
    print(macro_agg_df)
    # print(pd.pivot(agg_df, values="value", columns=[0], index=["class name"]))

#################### classification_from_segmentation-lr-3.549e-04.csv
    metric_name     value
126     F1Score  0.617829
127    Accuracy  0.628840
154     Jaccard  0.474331
155    Accuracy  0.713928
#################### classification_only-lr-3.336e-05.csv
    metric_name     value
154     F1Score  0.631912
155    Accuracy  0.628586
182     Jaccard  0.018651
183    Accuracy  0.065841
#################### fine_tuning-lr_init-1.63e-04-lr_backbone-2.88e-07.csv
    metric_name     value
120     F1Score  0.590983
121    Accuracy  0.606642
148     Jaccard  0.353025
149    Accuracy  0.630188
#################### multitask_simple-lr-9.160e-05.csv
    metric_name     value
140     F1Score  0.667116
141    Accuracy  0.661034
168     Jaccard  0.411100
169    Accuracy  0.679227
#################### segmentation_only-lr-1.268e-04.csv
    metric_name     value
126     F1Score  0.139001
127    Accuracy  0.171328
154     Jaccard  0.462900
155    Accuracy  0.718661


In [23]:
list(dfs.keys())
names =['classification from segmentation',
 'classification only',
 'finetuning',
 'multitask',
 'segmentation only'] 

In [24]:
macro_arr = np.vstack([df.value.values for df in macro_values])
macro_arr

array([[0.61782855, 0.62883973, 0.47433093, 0.71392769],
       [0.63191152, 0.62858605, 0.01865095, 0.06584086],
       [0.59098262, 0.60664153, 0.35302496, 0.63018811],
       [0.66711557, 0.66103399, 0.41109985, 0.67922688],
       [0.13900086, 0.17132796, 0.46290037, 0.71866071]])

In [25]:
def plot_bar(x, rows, names, xaxis_title,yaxis_title,title:str):
    fig = go.Figure(
    data=[
        *[go.Bar(name=name,x=x,y=row, text=[f"{item:.2f}" for item in row]) for name,row in zip(names,rows)]
    ]
)
# Change the bar mode
    fig.update_layout(barmode="group")
    fig.update_layout(
    autosize=False,
    width=1000,
    height=400,
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    template="seaborn",
)
    config = {
  'toImageButtonOptions': {
    'format': 'jpeg', # one of png, svg, jpeg, webp
    'filename': f"{title.replace(' ', '-')}",
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

    fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="right",
    x=1
))


    fig.show(config=config)

def get_values(regex, df_list):
    x = list([df.sort_values(by="class_name").loc[df.metric_name.str.contains(regex)].class_name.values for df in df_list][0])
    vals = [df.sort_values(by="class_name").loc[df.metric_name.str.contains(regex)].value.values for df in df_list]
    x = [item.replace("_", " ") for item in x]
    vals = np.vstack(vals)
    return x, vals

In [26]:
x = ["F1Score","Balanced Accuracy"]
names =['classification from segmentation',
 'classification only',
 'finetuning',
 'multitask',
 ] 
rows=macro_arr[:4,:2]
plot_bar(x, rows, names, "Metric Names", "Values", "Macro Classification")

In [27]:
x = ["IoU","Balanced Accuracy"]

names =[
 'finetuning',
 'multitask',
 'segmentation only',
 ] 

rows=macro_arr[2:,2:]
plot_bar(x, rows, names, "Metric Names", "Values", "Macro Segmentation")

In [28]:
x, rows = get_values("F1", classification_values)

names =['classification from segmentation',
 'classification only',
 'finetuning',
 'multitask',] 

plot_bar(x, rows[:4], names, "Class Names", "Values", "Classification F1Score")

In [29]:
x, rows = get_values("Accuracy", classification_values)

names =['classification from segmentation',
 'classification only',
 'finetuning',
 'multitask',] 

plot_bar(x, rows[:4], names, "Class Names", "Values", "Classification Accuracy")

In [30]:
x, rows = get_values("Accuracy", segmentation_values)

names =[
 'finetuning',
 'multitask',
 'segmentation only'] 

plot_bar(x, rows[2:], names, "Class Names", "Values", "Segmentation Accuracy")

In [31]:
x, rows = get_values("Jaccard", segmentation_values)

names =[
 'finetuning',
 'multitask',
 'segmentation only'] 

plot_bar(x, rows[2:], names, "Class Names", "Values", "Segmentation IoU")

In [24]:
from pathlib import Path

times = []
for root, dirs, filenames in os.walk(
    "/mnt/c/Users/piotr.hondra/Documents/inz/indoor-semantic-segmentation/logs_to_convert"
):
    # print(filenames)
    if "times.txt" in filenames:
        with open(os.path.join(root, "times.txt"), "r") as f:
            times.append(
                [
                    Path(root).parts[-2],
                    f"{float(f.readline().removeprefix('train,')):.2f}",
                ]
            )


In [28]:
for name, time in times:
    print(f"{name:<40}{time}")

classification_from_segmentation        70.78
classification_only                     170.47
fine_tuning                             158.46
multitask_simple                        210.97
segmentation_only                       188.70
